In [ ]:
# Global Variables
DATASET_IMAGES_DIR = "C:/Users/agand/ObjectDetectionFiles//logistics"
MODEL1_NAMES_FILE = "C:/Users/agand/ObjectDetectionFiles/yolo_model_1/logistics.names"
MODEL1_WEIGHTS = "C:/Users/agand/ObjectDetectionFiles/yolo_model_1/yolov4-tiny-logistics_size_416_1.weights"
MODEL1_CFG = "C:/Users/agand/ObjectDetectionFiles//yolo_model_1/yolov4-tiny-logistics_size_416_1.cfg"
MODEL2_NAMES_FILE = "C:/Users/agand/ObjectDetectionFiles/yolo_model_2/logistics.names"
MODEL2_WEIGHTS = "C:/Users/agand/ObjectDetectionFiles//yolo_model_2/yolov4-tiny-logistics_size_416_2.weights"
MODEL2_CFG = "C:/Users/agand/ObjectDetectionFiles//yolo_model_2/yolov4-tiny-logistics_size_416_2.cfg"
IMG_SIZE = (416, 416)
SCORE_THRESHOLD = 0.25
NMS_THRESHOLD = 0.45
OUTPUT_DIR = "C:/Users/agand/ObjectDetectionFiles/outputs"
MODEL1_OUT = "C:/Users/agand/ObjectDetectionFiles/outputs/model1"
MODEL2_OUT = "C:/Users/agand/ObjectDetectionFiles/outputs/model2"

In [ ]:
# Import libraries
import os, glob, json
import cv2
import numpy as np

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL1_OUT, exist_ok=True)  
os.makedirs(MODEL2_OUT, exist_ok=True)
IMAGES = glob.glob(os.path.join(DATASET_IMAGES_DIR, '*.jpg'))

In [ ]:
def yolo_model(cfgfile, weightfile,namesfile):
    net = cv2.dnn.readNet(weightfile,cfgfile)
    layers = net.getLayerNames()
    output_layers = [layers[i-1] for i in net.getUnconnectedOutLayers()]
    with open(namesfile, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    return net, output_layers, classes

net1, layers1, classes1 = yolo_model(MODEL1_CFG, MODEL1_WEIGHTS,MODEL1_NAMES_FILE)
net2, layers2, classes2 = yolo_model(MODEL2_CFG, MODEL2_WEIGHTS,MODEL2_NAMES_FILE)

In [ ]:
def predict_image(image_path, net, output_layers):
    image = cv2.imread(image_path)
    blob = cv2.dnn.blobFromImage(
        image,
        scalefactor=1/255.0,
        size=IMG_SIZE,
        mean=(0, 0, 0),
        swapRB=True,
        crop=False
    )
    net.setInput(blob)
    outputs = net.forward(output_layers)
    detections = []
    for feature_maps in outputs:
        for detection in feature_maps:
            boxes = detection[:4].tolist()
            score = detection[4].tolist()
            class_scores = detection[5:].tolist()
            class_id = int(np.argmax(class_scores))
            detections += [{"image_path":image_path,"class_id": class_id, "score": score, "box": boxes}]
    return detections

def get_ground_truth(image_path):
    gt_txt = os.path.splitext(image_path)[0] + ".txt"
    ground_truth = []
    if not os.path.exists(gt_txt):
        return ground_truth 

    with open(gt_txt, "r") as f:
        for line in f:
            parts = line.strip().split()
            cls = int(parts[0])
            ground_truth.append({
                "class_id": cls,
                "box": parts[1:5]
            })
    return ground_truth


def save_prediction_json(img_path,model,detections,output_json):
    json_data = {
        "image_path": img_path,
        "model":model,
        "predictions": detections,
        "ground_truth": get_ground_truth(img_path)
    }
    with open(output_json, 'w') as f:
        json.dump(json_data, f, indent=4)

for img_path in IMAGES:
    detections1 = predict_image(img_path, net1, layers1)
    detections2 = predict_image(img_path, net2, layers2)
    
    output_json1 = os.path.join(MODEL1_OUT, os.path.basename(img_path).replace('.jpg', '_predictions.json'))
    output_json2 = os.path.join(MODEL2_OUT, os.path.basename(img_path).replace('.jpg', '_predictions.json'))

    save_prediction_json(img_path,"model_1",detections1,output_json1)
    save_prediction_json(img_path,"model_2",detections2,output_json2)